In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import tensorflow_hub as hub
import random
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
tf.__version__

'2.15.0'

In [ ]:
!rm -rf /content/.cache && rm -rf /tmp/*
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/FaceData/data_test.pkl /content
!cp /content/drive/MyDrive/FaceData/data_train.pkl /content

In [ ]:
drive.flush_and_unmount()

In [ ]:
with open('data_test.pkl', 'rb') as f:
    test_df = pickle.load(f)

with open('data_train.pkl', 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
x_train = train_df['image']
y_train = train_df['person']

x_test = test_df['image']
y_test = test_df['person']

In [ ]:
def remove_extra_dim(image):
    try:
        return np.squeeze(image, axis=0)  # Efficient removal if size 1
    except ValueError:
        # Extra dimension not found, return the original data
        print('failed')
        return image

# Loop through each image in the Series
for index, image in x_train.items():  # Access elements using items()
    x_train.loc[index] = remove_extra_dim(image)  # Modify Series in-place


In [ ]:
for index, image in x_test.items():  # Access elements using items()
    x_test.loc[index] = remove_extra_dim(image)  # Modify Series in-place


In [ ]:
x_train[0].shape

(224, 224, 3)

In [ ]:
x_train.shape

(5574,)

In [ ]:
classes = set()
for i in y_train:
  classes.add(i)

classes = list(classes)
classes

['Abdoulaye Wade',
 'Hilary Duff',
 'Mohammed Baqir al-Hakim',
 'Luis Horna',
 'Gianna Angelopoulos-Daskalaki',
 'Gustavo Kuerten',
 'Steven Spielberg',
 'Lynn Redgrave',
 'Yashwant Sinha',
 'Aleksander Kwasniewski',
 'Ali Naimi',
 'Thomas Rupprath',
 'LeBron James',
 'Ari Fleischer',
 'Eric Robert Rudolph',
 'Grant Hackett',
 'Arianna Huffington',
 'Petria Thomas',
 'Johnny Tapia',
 'Tom Daschle',
 'Iain Duncan Smith',
 'Sonia Gandhi',
 'Diana Munz',
 'Antonio Banderas',
 'Bertie Ahern',
 'Serena Williams',
 'Nancy Pelosi',
 'Biljana Plavsic',
 'Brad Garrett',
 'Ron Dittemore',
 'Oswaldo Paya',
 'Amer al-Saadi',
 'Jack Nicholson',
 'Juan Manuel Marquez',
 'Christopher Walken',
 'Norm Coleman',
 'Kim Dae-jung',
 'Kwon Yang-sook',
 'Ricardo Lagos',
 'Tom Harkin',
 'Joe Lieberman',
 'Kate Winslet',
 'Paul Martin',
 'Vanessa Williams',
 'John Travolta',
 'Moshe Katsav',
 'Pedro Almodovar',
 'Jose Serra',
 'Matthew Perry',
 'Sylvester Stallone',
 'Naomi Watts',
 'Richard Gere',
 'Larry Cok

In [ ]:
def data_generator(batch_size=64):
  while True:
    a = []
    p = []
    n = []
    for _ in range(batch_size):
      found = True
      while(found):
        pos_neg = random.sample(classes, 2)
        try:
          positive_samples = random.sample(list (x_train [y_train==pos_neg [0]]), 2)
          negative_sample= random.choice(list (x_train [y_train == pos_neg [1]]))
          found = False
        except:
          continue
      a.append(positive_samples [0])
      p.append(positive_samples [1])
      n.append(negative_sample)

    yield ([np.array(a), np.array(p), np.array(n)], np.zeros((batch_size, 1)).astype("float32"))

In [ ]:
def triplet_loss (y_true, y_pred):
  anchor_out = y_pred[:, 0:128]
  positive_out = y_pred[:, 128:256]
  negative_out = y_pred[:, 256:384]
  pos_dist = K.sum(K.abs(anchor_out-positive_out), axis=1)
  neg_dist = K.sum(K.abs(anchor_out - negative_out), axis=1)
  stacked_distances = K.stack([pos_dist, neg_dist], axis=0)
  probs = K.softmax(stacked_distances, axis=0)
  return K.mean(K.abs (probs [0]) + K.abs (1.0 - probs [1]))

In [ ]:
input_shape = (224,224,3);
input_layer = layers.Input(shape=input_shape)
inception = InceptionV3(weights='imagenet', include_top=False)
feature_extractor = inception(input_layer)
for layer in inception.layers:
    layer.trainable = False
gap_layer = layers.GlobalAveragePooling2D()(feature_extractor)
fc_layer = layers.Dense(128, activation='relu')(gap_layer)
l2_norm_layer = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(fc_layer)
model = Model(inputs=input_layer, outputs=l2_norm_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, None, None, 2048   21802784  
                             )                                   
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 lambda (Lambda)             (None, 128)               0         
                                                                 
Total params: 22065056 (84.17 MB)
Trainable params: 262272 (1

In [ ]:
triplet_model_a = layers.Input((224, 224, 3))
triplet_model_p = layers.Input((224, 224, 3))
triplet_model_n = layers.Input((224, 224, 3))
triplet_model_out = layers.Concatenate()([model (triplet_model_a), model (triplet_model_p), model(triplet_model_n)])
triplet_model = Model( [triplet_model_a, triplet_model_p, triplet_model_n], triplet_model_out)
# triplet_model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

In [ ]:
triplet_model.compile(loss=triplet_loss, optimizer=optimizer )

In [ ]:
triplet_model.fit_generator(data_generator(),steps_per_epoch=87,epochs=200)

<ipython-input-17-619d56eeb80a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  triplet_model.fit_generator(data_generator(),steps_per_epoch=87,epochs=200)


Epoch 1/200
 5/87 [>.............................] - ETA: 33:46 - loss: 0.6943

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.
classes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]

In [ ]:
def data_generator(batch_size=64):
    while True:
        a = []
        p = []
        n = []
        for _ in range(batch_size):
            pos_neg = random.sample(classes, 2)
            positive_samples = random.sample(list(x_train[y_train == pos_neg[0]]), 2)
            negative_sample = random.choice(list(x_train[y_train == pos_neg[1]]))
            a.append(positive_samples[0])
            p.append(positive_samples[1])
            n.append(negative_sample)
        yield ([np.array(a), np.array(p), np.array(n)], np.zeros((batch_size, 1)).astype("float32"))

In [ ]:
def triplet_loss(y_true, y_pred):
    anchor_out = y_pred[:, 0:100]
    positive_out = y_pred[:, 100:200]
    negative_out = y_pred[:, 200:300]

    pos_dist = K.sum(K.abs(anchor_out - positive_out), axis=1)
    neg_dist = K.sum(K.abs(anchor_out - negative_out), axis=1)
    stacked_distances = K.stack([pos_dist, neg_dist], axis=0)
    probs = K.softmax(stacked_distances, axis=0)


    return K.mean(K.abs(probs[0]) + K.abs(1.0 - probs[1]))

In [ ]:
input_layer = Input((28, 28, 1))
x = Conv2D(32, 3, activation="relu")(input_layer)
x = Conv2D(32, 3, activation="relu")(x)
x = MaxPool2D(2)(x)
x = Conv2D(64, 3, activation="relu")(x)
x = Conv2D(64, 3, activation="relu")(x)
x = MaxPool2D(2)(x)
x = Conv2D(128, 3, activation="relu")(x)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
model = Model(input_layer, x)
model.summary()

In [ ]:
triplet_model_a = Input((28, 28, 1))
triplet_model_p = Input((28, 28, 1))
triplet_model_n = Input((28, 28, 1))
triplet_model_out = Concatenate()([model(triplet_model_a), model(triplet_model_p), model(triplet_model_n)])
triplet_model = Model([triplet_model_a, triplet_model_p, triplet_model_n], triplet_model_out)
triplet_model.summary()

In [ ]:
triplet_model.compile(loss=triplet_loss, optimizer="adam")

In [ ]:
import csv


# Example usage
data_file = 'your_data.txt'  # Replace with your actual file path
output_file = 'data.csv'
convert_to_csv(data_file, output_file)
print(f'Converted data to CSV: {output_file}')


In [ ]:
triplet_model.fit_generator(data_generator(), steps_per_epoch=150, epochs=3)